In [ ]:
import requests
import json
import sys
import numpy as np
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn as sk
import torch
import test_utils_models
from importlib import reload
from torch.utils.data import DataLoader, TensorDataset
import Transformer
import TransformerV3
import TransformerService
import TransformerServiceV2
import torch.optim as optim
import trainer.Trainer as Trainer
import torch.nn as nn
import test_utils
import DataVisualizations
import Utilities
import models.BuiltModels as BuiltModels
import models.decoders
import models.encoders
import models.ModelScraper
import models

In [ ]:
url = 'http://localhost:8000/sequenceset_manager/get_sequence_data'

features = ['pctChgclose', 'pctChgvolume', 'opHi', 'opCl','hiCl','loCl','opLo',"pctDiff+ema5_close", "pctDiff+ema10_close", "pctDiff+ema20_close",'pctDiff+ema50_close', 'pctDiff+ema100_close',
'pctDiff+ema200_close','pctDiff+bb_high_close10', 'pctDiff+bb_low_close10', 'pctDiff+smaVol10_volume', 'pctDiff+smaVol20_volume', 'pctChgClOp', 'macd_signal','bb_indicator20',"bb_indicator50",
            ]
# struct_features = ['open', 'high', 'low', 'close', 'ema20', 'ema50', 'ema100']
# struct_features = ['close']
struct_features = []
# features = ['pctChgclose','pctChgvolume','opHi', 'opCl','hiCl','loCl','opLo']
# features = ['pctChgclose']
# features = ['close']
# target_features = ['pctChgclose+{}'.format(i) for i in range(1, 16) ]
# target_features = ['close+{}'.format(i) for i in range(1, 16) ]
target_features = ['cumPctChg+{}'.format(i) for i in range(1,16) ]

indices_seq = list(range(len(features + struct_features + target_features)))

feature_dict = {col: index for col, index in zip(features+struct_features + target_features, indices_seq)}


In [ ]:
print(feature_dict)

In [ ]:
# tickers = ['MSFT', 'AAPL', 'AMZN','QQQ','SPY','F', 'TSLA', "BAC", "LLY", "PEP" ]
# tickers = ['AAPL', 'MSFT','AMZN','QQQ','SPY']
tickers = ["AAPL", "SPY", "QQQ", "XOM", "MSFT", "AMZN", "BB", 'F', 'TSLA', 'GE',
           "JPM", "BAC", "LLY", "NKE", "WMT", "PEP"]
sequence_lengths = [50]
all_sequences = []

all_params = []

for ticker in tickers:
    for sequence_length in sequence_lengths:
        all_params.append({
            'ticker': ticker,
            'features': features + struct_features + target_features,
            'interval': '1d',
            'start_date': '2007-01-01',
            'sequence_length': sequence_length
        })
for params in all_params:

    response = requests.get(url, params=params)
    if response.status_code == 200:
        try:
            data = response.json()
            all_sequences.append(data)
        except json.JSONDecodeError as e:
            print(f"Failed to decode JSON: {e}")
            print("Response text:", response.text)
    else:
        print(f"Request failed with status code {response.status_code}")

In [ ]:
print(len(all_sequences))

In [ ]:
print(all_sequences[-1][-500]['end_timestamp'])
print(all_sequences[-1][-500]['start_timestamp'])


In [ ]:
print(all_sequences[-1][-500]['sliced_data'][-1][feature_dict['pctChgclose']])

In [ ]:
target_feauture_index = [feature_dict[col] for col in target_features]
for index in target_feauture_index:
    print(all_sequences[-1][-500]['sliced_data'][-1][index])

In [ ]:
def create_3d_array(stock_sequences, max_len_seq):
    # Create a 3D array to store the sequences
    # The array will have the shape (num_sequences, sequence_length, num_features)
    # stock sequences is a list of sequence_dicts where each sequence_dict is a dictionary
    # we are extracting 'sliced_data' from each sequence_dict and creating 3d np array

    num_features = len(features) + len(struct_features)

    sequence_steps = len(stock_sequences[0]['sliced_data'])

    num_sequences = len(stock_sequences)
    X = np.zeros((num_sequences, sequence_steps, num_features))
    y = np.zeros((num_sequences, len(target_features)))

    indices_map = {}
    cur_index = 0
    for i, sequence_dict in enumerate(stock_sequences):
        X_cols = [feature_dict[col] for col in features + struct_features]
        y_cols = [feature_dict[col] for col in target_features]

        X[i, :, :] = np.array(sequence_dict['sliced_data'])[:, X_cols]
        X_padded = np.zeros((num_sequences, max_len_seq, num_features))
        X_padded[:, :sequence_steps, :] = X
        y[i, :] = np.array(sequence_dict['sliced_data'])[-1, y_cols]




    return X_padded, y

In [ ]:
def preprocess_multi_stock(all_sequences, max_seq_length, test_size=0.1):
    X_train = np.zeros((0, 0, 0))
    X_test = np.zeros((0, 0, 0))
    y_train = np.zeros((0, 0))
    y_test = np.zeros((0, 0))
    
    test_seq_objs = []
    train_seq_objs = []
    for sequences in all_sequences:
        print(len(sequences))
        X, y = create_3d_array(sequences,max_seq_length)
        print(X.shape)

        for i in range(len(X)-1, 0, -1):
            if np.isnan(y[i]).any():
                X = np.delete(X, i, axis=0)
                y = np.delete(y, i, axis=0)

        X_train_cur, X_test_cur, y_train_cur, y_test_cur = train_test_split(X, y, test_size=test_size, shuffle=False)
        #TODO need to set train/test start end date and split on time instead of percentage of dataset size
        
        train_seq_objs += [sequences[i] for i in range(len(X_train_cur))]
        test_seq_objs += [sequences[i] for i in range(len(X_train_cur), len(X_train_cur)+len(X_test_cur))]

        # scaler = TimeSeriesScalerMeanVariance(mu=0., std=1.)
        # X_train_cur = scaler.fit_transform(X_train_cur)
        # X_test_cur = scaler.transform(X_test_cur)
        # 
        y_scaler = sk.preprocessing.StandardScaler()
        # y_train_cur = y_scaler.fit_transform(y_train_cur)
        # y_test_cur = y_scaler.transform(y_test_cur)
        
        # X_train_cur = X_train_cur[0:-25, :, :]
        # y_train_cur = y_train_cur[0:-25, :]
        # X_test_cur = X_test_cur[0:-25, :, :]
        # y_test_cur = y_test_cur[0:-25, :]


        if X_train.shape[0] == 0:
            X_train = X_train_cur
            X_test = X_test_cur
            y_train = y_train_cur
            y_test = y_test_cur
        else:
            X_train = np.concatenate((X_train, X_train_cur), axis=0)
            X_test = np.concatenate((X_test, X_test_cur), axis=0)
            y_train = np.concatenate((y_train, y_train_cur), axis=0)
            y_test = np.concatenate((y_test, y_test_cur), axis=0)

    return X_train, X_test, y_train, y_test, y_scaler, train_seq_objs, test_seq_objs


In [ ]:
X_train_whole, X_test, y_train_whole, y_test, y_scaler,train_seq_objs, test_seq_objs = preprocess_multi_stock(all_sequences, max(sequence_lengths))

In [ ]:
def sample_training_data(X_train, y_train, percentage=0.3):
    # Get the number of samples
    num_samples = X_train.shape[0]

    # Calculate the number of samples for 30%
    num_samples_to_sample = int(percentage * num_samples)

    # Randomly sample indices (without replacement)
    sampled_indices = np.random.choice(num_samples, num_samples_to_sample, replace=False)

    # Extract the same indices from both X_train and y_train
    X_sampled = X_train[sampled_indices]
    y_sampled = y_train[sampled_indices]

    return X_sampled, y_sampled

In [ ]:
## sample random 30% of the sequences for training set and get rid of the rest
# X_train, y_train = sample_training_data(X_train_whole, y_train_whole, percentage=1)
X_train = X_train_whole
y_train = y_train_whole

In [ ]:


print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
print(feature_dict)

In [ ]:
if len(struct_features) > 0:
    X_train_scaled = Utilities.scale_sequence_by_group(struct_features, feature_dict, X_train)
    X_test_scaled = Utilities.scale_sequence_by_group(struct_features, feature_dict, X_test)
else:
    X_train_scaled = X_train
    X_test_scaled = X_test

In [ ]:
x_indices = [feature_dict[col] for col in features]

scaler = TimeSeriesScalerMeanVariance(mu=0., std=1.)
X_train_scaled[:,:,x_indices] = scaler.fit_transform(X_train[:,:,x_indices])
X_test_scaled[:,:,x_indices] = scaler.transform(X_test[:,:,x_indices])

y_train_scaled = y_scaler.fit_transform(y_train)
y_test_scaled = y_scaler.transform(y_test)
# X_train_scaled = X_train
# X_test_scaled = X_test
# y_train_scaled = y_train
# y_test_scaled = y_test

In [ ]:
y_train_scaled = np.expand_dims(y_train_scaled, axis=2)
y_test_scaled = np.expand_dims(y_test_scaled, axis=2)
print(y_train_scaled.shape, y_test_scaled.shape)

In [ ]:
print(X_train_scaled.shape, X_test_scaled.shape)

In [ ]:
print(X_train_scaled[0,:,-1])

In [ ]:
DataVisualizations.plot_feature_histograms(X_test_scaled, features + struct_features, feature_dict)


In [ ]:
DataVisualizations.plot_y_feature_histograms(y_train_scaled, target_features)

In [ ]:
DataVisualizations.plot_y_feature_histograms(y_test_scaled, target_features)

In [ ]:
DataVisualizations.plot_y_feature_histograms(np.expand_dims(y_scaler.inverse_transform(y_train_scaled.squeeze()),-1), target_features)


In [ ]:
DataVisualizations.plot_y_feature_histograms(np.expand_dims(y_train,-1), target_features)


In [ ]:
reload(DataVisualizations)

In [ ]:
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")

else:
    print("MPS is available. Setting as default device.")
    device = torch.device("mps")
    torch.set_default_device(device)

print(f"Using device: {device}")
batch_size = 64

# Convert data to tensors and move them to the appropriate device
print(X_train_scaled.shape)
print(y_train_scaled.shape)
# X_train_scaled_concat = np.concatenate((X_train_scaled, y_train_scaled), axis=-1)
# print(X_train_scaled_concat.shape)
# X_test_scaled_concat = np.concatenate((X_test_scaled, y_test_scaled), axis=-1)
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test_scaled, dtype=torch.float32).to(device)

print(X_train_tensor.device, X_test_tensor.device, y_train_tensor.device, y_test_tensor.device)

# Create TensorDataset and DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

generator = torch.Generator(device=device)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, generator=generator)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
print(X_train_tensor.shape, y_train_tensor.shape, X_test_tensor.shape, y_test_tensor.shape)

In [ ]:
num_layers = 2
d_model = 32
num_heads = 2
d_ff = 64
input_dim = X_train_tensor.shape[2]  # Number of features in X_train
# output_dim = 1  # Since y_train has shape (batch, output_steps, 1)
output_dim = y_train_tensor.shape[1]
# Initialize the Transformer models
model_obj = BuiltModels.Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    d_ff=d_ff,
    encoder_input_dim=input_dim,
    decoder_input_dim=output_dim,
    dropout=.2
).to(device)
print(next(model_obj.parameters()).device)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

print(f'The model has {count_parameters(model_obj):,} trainable parameters')

In [ ]:
optimizer = optim.Adam(model_obj.parameters(), lr=0.001)
# base_optimizer = optim.Adam
# optimizer = torch.optim.SGD(models.parameters(), lr=1e-3, momentum=0.9)

# optimizer = Transformer.SAM(models.parameters(), base_optimizer, rho=0.3, lr=0.01)
# criterion = nn.MSELoss()
# get mae criterian 
# criterion = nn.L1Loss()
# criterion = nn.HuberLoss(delta=1.0)
criterion = Transformer.gaussian_nll_loss
# Initialize the Trainer with the device
trainer_obj = Trainer.Trainer(
    model=model_obj,
    optimizer=optimizer,
    criterion=criterion,
    device=device,
    max_length=2,
    start_token_value=-999 # Start token value, adjust if necessary
)

In [ ]:
from importlib import reload
reload(BuiltModels)
reload(models.decoders)
reload(models.encoders)
reload(models.ModelScraper)
reload(Trainer)


In [ ]:
# file_path = "saved_models/model_weights20240929_113227.pth"
# model_obj.load_state_dict(torch.load(file_path, map_location=device))

In [ ]:
epochs = 250

# Train the models

trainer_obj.fit(train_dataloader, test_dataloader, epochs=epochs, clip_value=None)

Epoch 1 | Train Loss: 0.850 | Val Loss: 1.922
Epoch 2 | Train Loss: 0.851 | Val Loss: 1.890
Epoch 3 | Train Loss: 0.852 | Val Loss: 1.948
Epoch 4 | Train Loss: 0.852 | Val Loss: 1.899


In [ ]:
gradient_test_X = X_train_tensor[-64:]
gradient_test_y = y_train_tensor[-64:]

gradients = trainer_obj.log_gradients_with_names(gradient_test_X, gradient_test_y)
DataVisualizations.visualize_gradients(gradients)

In [ ]:
# y_pred = trainer_obj.predict(X_test_tensor[:256])
y_pred = trainer_obj.predict(X_test_tensor)
print(y_pred.shape)

In [ ]:
scaling_factor = 1
y_pred = y_pred * scaling_factor
mu_pred = y_pred[:, :, 0]
log_var_pred = y_pred[:, :, 1]
variance_pred = torch.exp(log_var_pred)
std_pred = torch.sqrt(variance_pred)
print(mu_pred.shape, std_pred.shape)


In [ ]:
mu_pred_denorm = y_scaler.inverse_transform(mu_pred.detach().cpu().numpy())
std_pred_denorm = y_scaler.inverse_transform(std_pred.detach().cpu().numpy())
# mu_pred_denorm = mu_pred.detach().cpu().numpy()
# std_pred_denorm = std_pred.detach().cpu().numpy()

In [ ]:
trainer_obj.generate_model_architecture()

In [ ]:
# retrieval_list = ["transformer_encoder:encoder_layer0:multi_head_attention",
# "transformer_encoder:encoder_layer1:multi_head_attention",
# "transformer_decoder:decoder_layer0:self_attention",
# "transformer_decoder:decoder_layer0:cross_attention",
# "transformer_decoder:decoder_layer1:self_attention",
# "transformer_decoder:decoder_layer1:cross_attention"]
retrieval_list = [
    'transformer_encoder:encoder_layer0',
    'transformer_encoder:encoder_layer0:multi_head_attention',
    'transformer_encoder:encoder_layer1',
    'transformer_encoder:encoder_layer1:multi_head_attention',
    'DecoderRNN'

]
output_dict = trainer_obj.scrape_model(retrieval_list)

In [ ]:
for key in output_dict.keys():
    print(key)
    print(output_dict[key].keys())

In [ ]:
# print(output_dict['transformer_encoder:encoder_layer0:multi_head_attention'].shape)
print(len(output_dict))

In [ ]:
print(y_pred.shape)

In [ ]:
reload(test_utils)
reload(Transformer)
reload(TransformerService)
reload(DataVisualizations)
reload(BuiltModels)
reload(Trainer)
reload(models.encoders)
reload(models.ModelScraper)
reload(models.decoders)

In [ ]:
mu_pred = mu_pred_denorm.squeeze()
std_pred = std_pred_denorm.squeeze()

In [ ]:
print(mu_pred.shape)

In [ ]:
output_string, results, predicted_y_old = test_utils.eval_model(y_test, mu_pred, num_days = y_pred.shape[1], alreadyCum=True)

In [ ]:
print(output_string)

In [ ]:
test_utils.visualize_future_distribution(results)

In [ ]:
DataVisualizations.visualize_step_distribution(mu_pred,y_test)

In [ ]:
# visualize variance
DataVisualizations.visualize_std(std_pred)

In [ ]:
DataVisualizations.plot_residuals(y_test ,mu_pred, std_pred)

In [ ]:
DataVisualizations.plot_layer_activations(model_obj, X_test_tensor)

In [215]:
for i in range(500, 510):

    # print(test_seq_objs[i]['ticker'])
    print(test_seq_objs[i]['start_timestamp'])
    print(test_seq_objs[i]['end_timestamp'])
    print(y_test[i])
    DataVisualizations.layer_heatmap(output_dict, i)
    DataVisualizations.visualize_future_predictions_with_uncertainty(y_train[i], mu_pred[i], std_pred[i], already_cumulative=True)

2022-12-07T00:00:00Z
2023-02-17T00:00:00Z


KeyError: 'ticker'

In [ ]:
# embeddings = DataVisualizations.extract_embeddings(models, X_test_tensor[:1000])
# DataVisualizations.visualize_embeddings(embeddings, method='tsne', perplexity=50)

In [ ]:
from datetime import datetime

# add date/time to models name
model_name = 'model_weights' + datetime.now().strftime("%Y%m%d_%H%M%S")
torch.save(model_obj.state_dict(), 'saved_models/' + model_name + '.pth')